In [2]:
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-ix3pj6zm
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-ix3pj6zm
     |████████████████████████████████| 50 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 9.8 MB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.6.2-cp37-cp37m-linux_x86_64.whl size=31526579 sha256=f43ab2e0cdad0ee4aa3ada9eda0c54cdd05ec97d57e75e38092a5a06cfded416
  Stored in directory: /tmp/pip-ephem-wheel-cache-hy6qfy3x/wheels/db/2f/07/b84807ee4c9ffc917b90b716977d7fea8f9e841f6173150600
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=d11130ccab01a4a85cbcd2d283a842e8d08236b1222e60cf0083d97c27c59ac4
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pyt

In [3]:
import os
import numpy as np
import json

from PIL import Image  
from csv import writer
import csv

from tensorflow import keras
import tensorflow as tf

from typing import Tuple

# import torch
# import torch.nn.functional as F
# from pytorch3d import _C
# from torch.autograd import Function

from pytorch3d.ops import box3d_overlap
import torch

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun  8 06:59:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
WIDTH = 730
HEIGHT = 530
CHANNELS = 3

BATCH_SIZE = 8
TRAIN_DATASET_NO = 3136

TEST_DATASET_NO = 348

SAMPLES_NO = TRAIN_DATASET_NO // BATCH_SIZE

DATA_PATH = "datasets/SUNRGB-D3D/SUNRGBD/kv2/kinect2data"
CSV_EXISTS = True


X_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_images_k2.csv'
y_TRAIN_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_annotations_k2.csv'
X_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_images_k2.csv'
y_TEST_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_annotations_k2.csv'
X_PREDICT_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/predict_k2.csv'

y_TRAIN_3DBB_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/train_annotations_3dbb_k2.csv'
y_TEST_3DBB_DATA_PATH = '/content/drive/MyDrive/Colab Datasets/test_annotations_3dbb_k2.csv'


In [6]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

from typing import Tuple

import torch
import torch.nn.functional as F
from pytorch3d import _C
from torch.autograd import Function


# -------------------------------------------------- #
#                  CONSTANTS                         #
# -------------------------------------------------- #
"""
_box_planes and _box_triangles define the 4- and 3-connectivity
of the 8 box corners.
_box_planes gives the quad faces of the 3D box
_box_triangles gives the triangle faces of the 3D box
"""
_box_planes = [
    [0, 1, 2, 3],
    [3, 2, 6, 7],
    [0, 1, 5, 4],
    [0, 3, 7, 4],
    [1, 2, 6, 5],
    [4, 5, 6, 7],
]
_box_triangles = [
    [0, 1, 2],
    [0, 3, 2],
    [4, 5, 6],
    [4, 6, 7],
    [1, 5, 6],
    [1, 6, 2],
    [0, 4, 7],
    [0, 7, 3],
    [3, 2, 6],
    [3, 6, 7],
    [0, 1, 5],
    [0, 4, 5],
]


def _check_coplanar(boxes: torch.Tensor, eps: float = 1e-4) -> None:
    faces = torch.tensor(_box_planes, dtype=torch.int64, device=boxes.device)
    # pyre-fixme[16]: `boxes` has no attribute `index_select`.
    verts = boxes.index_select(index=faces.view(-1), dim=1)
    B = boxes.shape[0]
    P, V = faces.shape
    # (B, P, 4, 3) -> (B, P, 3)
    v0, v1, v2, v3 = verts.reshape(B, P, V, 3).unbind(2)

    # Compute the normal
    e0 = F.normalize(v1 - v0, dim=-1)
    e1 = F.normalize(v2 - v0, dim=-1)
    normal = F.normalize(torch.cross(e0, e1, dim=-1), dim=-1)

    # Check the fourth vertex is also on the same plane
    mat1 = (v3 - v0).view(B, 1, -1)  # (B, 1, P*3)
    mat2 = normal.view(B, -1, 1)  # (B, P*3, 1)
    if not (mat1.bmm(mat2).abs() < eps).all().item():
        msg = "Plane vertices are not coplanar"
        raise ValueError(msg)

    return


def _check_nonzero(boxes: torch.Tensor, eps: float = 1e-4) -> None:
    """
    Checks that the sides of the box have a non zero area
    """
    faces = torch.tensor(_box_triangles, dtype=torch.int64, device=boxes.device)
    # pyre-fixme[16]: `boxes` has no attribute `index_select`.
    verts = boxes.index_select(index=faces.view(-1), dim=1)
    B = boxes.shape[0]
    T, V = faces.shape
    # (B, T, 3, 3) -> (B, T, 3)
    v0, v1, v2 = verts.reshape(B, T, V, 3).unbind(2)

    normals = torch.cross(v1 - v0, v2 - v0, dim=-1)  # (B, T, 3)
    face_areas = normals.norm(dim=-1) / 2

    if (face_areas < eps).any().item():
        msg = "Planes have zero areas"
        raise ValueError(msg)

    return


class _box3d_overlap(Function):
    """
    Torch autograd Function wrapper for box3d_overlap C++/CUDA implementations.
    Backward is not supported.
    """

    @staticmethod
    def forward(ctx, boxes1, boxes2):
        """
        Arguments defintions the same as in the box3d_overlap function
        """
        vol, iou = _C.iou_box3d(boxes1, boxes2)
        return vol, iou

    @staticmethod
    def backward(ctx, grad_vol, grad_iou):
        raise ValueError("box3d_overlap backward is not supported")


def my_box3d_overlap(
    boxes1: torch.Tensor, boxes2: torch.Tensor, eps: float = 1e-4
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Computes the intersection of 3D boxes1 and boxes2.

    Inputs boxes1, boxes2 are tensors of shape (B, 8, 3)
    (where B doesn't have to be the same for boxes1 and boxes1),
    containing the 8 corners of the boxes, as follows:

        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)


    NOTE: Throughout this implementation, we assume that boxes
    are defined by their 8 corners exactly in the order specified in the
    diagram above for the function to give correct results. In addition
    the vertices on each plane must be coplanar.
    As an alternative to the diagram, this is a unit bounding
    box which has the correct vertex ordering:

    box_corner_vertices = [
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1],
    ]

    Args:
        boxes1: tensor of shape (N, 8, 3) of the coordinates of the 1st boxes
        boxes2: tensor of shape (M, 8, 3) of the coordinates of the 2nd boxes
    Returns:
        vol: (N, M) tensor of the volume of the intersecting convex shapes
        iou: (N, M) tensor of the intersection over union which is
            defined as: `iou = vol / (vol1 + vol2 - vol)`
    """
    if not all((8, 3) == box.shape[1:] for box in [boxes1, boxes2]):
        raise ValueError("Each box in the batch must be of shape (8, 3)")

    # pyre-fixme[16]: `_box3d_overlap` has no attribute `apply`.
    vol, iou = _box3d_overlap.apply(boxes1, boxes2)

    return vol, iou


In [7]:
# 3D IoU Custom Metric

def iou_3D(y_true, y_pred):

    # Convert form tf tensor to numpy
    y_true_np = y_true.numpy().reshape(-1, 8, 2).astype(np.float32)
    y_pred_np = y_pred.numpy().reshape(-1, 8, 2).astype(np.float32)

    # Add the fictive third dimension
    y_true_np = np.append(y_true_np, np.zeros((y_true_np.shape[0], y_true_np.shape[1], 1)), axis=2)
    y_pred_np = np.append(y_pred_np, np.zeros((y_pred_np.shape[0], y_pred_np.shape[1], 1)), axis=2)


    # Reorder points to be compatible with pytorch3D requirements

    # TODO
    y_true_np_reorder = np.zeros(y_true_np.shape)
    y_pred_np_reorder = np.zeros(y_pred_np.shape)
    y_true_np_reorder[:, 0, :] = y_true_np[:, 2, :]
    y_pred_np_reorder[:, 0, :] = y_pred_np[:, 2, :]
    y_true_np_reorder[:, 1, :] = y_true_np[:, 3, :]
    y_pred_np_reorder[:, 1, :] = y_pred_np[:, 3, :]
    y_true_np_reorder[:, 2, :] = y_true_np[:, 7, :]
    y_pred_np_reorder[:, 2, :] = y_pred_np[:, 7, :]
    y_true_np_reorder[:, 3, :] = y_true_np[:, 6, :]
    y_pred_np_reorder[:, 3, :] = y_pred_np[:, 6, :]
    y_true_np_reorder[:, 4, :] = y_true_np[:, 1, :]
    y_pred_np_reorder[:, 4, :] = y_pred_np[:, 1, :]
    y_true_np_reorder[:, 5, :] = y_true_np[:, 0, :]
    y_pred_np_reorder[:, 5, :] = y_pred_np[:, 0, :]
    y_true_np_reorder[:, 6, :] = y_true_np[:, 4, :]
    y_pred_np_reorder[:, 6, :] = y_pred_np[:, 4, :]
    y_true_np_reorder[:, 7, :] = y_true_np[:, 5, :]
    y_pred_np_reorder[:, 7, :] = y_pred_np[:, 5, :]
    ''' 
            1 -------- 0
           /|         /|
          2 -------- 3 .
          | |        | |
          . 5 -------- 4
          |/         |/
          6 -------- 7
                |
                V
        (4) +---------+. (5)
            | ` .     |  ` .
            | (0) +---+-----+ (1)
            |     |   |     |
        (7) +-----+---+. (6)|
            ` .   |     ` . |
            (3) ` +---------+ (2)
    '''

    # Add the 1 value for "back" points
    y_true_np_reorder[:, 4:, :] = 1
    y_pred_np_reorder[:, 4:, :] = 1

    # Scale the value back to original dimensions
    y_true_np_reorder[:, :, 0] *= WIDTH
    y_true_np_reorder[:, :, 1] *= HEIGHT
    y_pred_np_reorder[:, :, 0] *= WIDTH
    y_pred_np_reorder[:, :, 1] *= HEIGHT

    # Convert from numpy to tf tensor
    y_true_torch_init = torch.tensor(y_true_np_reorder)
    y_pred_torch_init = torch.tensor(y_pred_np_reorder)

    # Reshape to (None, 8, 3) dimension
    y_true_torch_reshaped = torch.reshape(y_true_torch_init, (-1, 8, 3)).type(torch.float32)
    y_pred_torch_reshaped = torch.reshape(y_pred_torch_init, (-1, 8, 3)).type(torch.float32)


    # _, iou_3d = box3d_overal(torch_y_true, torch_y_pred)
    _, iou_3d = my_box3d_overlap(y_true_torch_reshaped, y_pred_torch_reshaped, 0.0)

    return iou_3d
    # return 0


In [8]:
def csv_lazy(file_name):
    with open(file_name) as f:
        r = csv.reader(f)
        
        for row in r:
            yield row

def batch_generator(X_file, batch_size):

    csv_generator = csv_lazy(X_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, np.array(next(csv_generator)).astype('float') / 255.0)
        
        yield batch.reshape(batch_size, WIDTH, HEIGHT, CHANNELS).astype('float')

def label_generator(y_file, batch_size):
    csv_generator = csv_lazy(y_file)

    while True:
        batch = np.array([])

        for i in range(batch_size):
            batch = np.append(batch, next(csv_generator))
        
        yield batch.reshape(batch_size, 16).astype('float')
        
def model_generator(X_file, y_file, batch_size, limit_iteration):
    X_generator = batch_generator(X_file, batch_size)
    y_generator = label_generator(y_file, batch_size)
    
    iteration_no = 0

    while True:
        
        iteration_no += 1

        print('Iteration: ' + str(iteration_no))

        # Reset to start of file
        if iteration_no > limit_iteration:
            X_generator = batch_generator(X_file, batch_size)
            y_generator = label_generator(y_file, batch_size)
            iteration_no = 0

        X_batch = next(X_generator)
        y_batch = next(y_generator)

        yield (X_batch, y_batch)

In [9]:
base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(16)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=[iou_3D], run_eagerly=True)

9420800/9406464 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [11]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_3DBB_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 10),
          steps_per_epoch = SAMPLES_NO // 10,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
39/39 - 205s - loss: 0.1404 - iou_3D: 0.2511 - 205s/epoch - 5s/step
Epoch 2/10
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iter

In [12]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_3DBB_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

[0.3247320055961609, 0.09268251806497574]

In [13]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 0s - 136ms/epoch - 136ms/step


array([[ 1.0862429 ,  0.1532132 ,  0.7326556 , -0.50588745, -0.3325705 ,
         0.6789566 ,  0.10229226, -0.17238565,  0.7408254 ,  0.19129132,
         1.5304009 ,  0.47896492,  0.1815678 ,  1.065481  ,  0.8376928 ,
        -0.20529953],
       [ 1.0519305 ,  0.18296233,  0.9682177 , -0.3502861 , -0.34140837,
         0.6291793 ,  0.19951423,  0.382872  ,  0.961182  ,  0.68660426,
         1.7663678 ,  0.61595035, -0.26665604,  0.838686  ,  0.8961112 ,
        -0.2561931 ],
       [ 1.3362414 , -0.1974436 ,  1.3040631 , -0.3783395 , -0.59979147,
         1.2786034 ,  0.3961884 ,  0.06464918,  1.4947239 ,  0.49217418,
         0.826311  ,  1.0302603 ,  0.11118528,  1.490939  ,  0.66266495,
         0.09705059],
       [ 1.4852413 ,  0.7174995 ,  0.81585205, -0.19998124, -0.08325712,
         0.5562734 ,  0.272997  , -0.5164143 ,  0.5138887 ,  0.7732892 ,
         1.4504392 ,  0.35023898,  0.15596494,  0.625667  ,  0.67049915,
         0.10413037],
       [ 2.061818  ,  0.23396602,  1

In [14]:
base_model = keras.applications.MobileNetV2(weights='imagenet', include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
loc_output = keras.layers.Dense(16)(avg)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)

model = keras.models.Model(inputs=base_model.input, outputs=loc_output)
model.compile(loss='mse', optimizer=optimizer, metrics=[iou_3D], run_eagerly=True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [15]:
model.fit(x = model_generator(X_TRAIN_DATA_PATH, y_TRAIN_3DBB_DATA_PATH, BATCH_SIZE, SAMPLES_NO // 4),
          steps_per_epoch = SAMPLES_NO // 4,
          epochs = 10,
          verbose = 2
          )

Iteration: 1
Epoch 1/10
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iter

In [16]:
model.evaluate(
    x = model_generator(X_TEST_DATA_PATH, y_TEST_3DBB_DATA_PATH, 4, TEST_DATASET_NO // 4),
    verbose = 2,
    steps = TEST_DATASET_NO // 4 - 1
)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

[0.1598643809556961, 0.2588276267051697]

In [17]:
model.predict(
    x = batch_generator(X_PREDICT_DATA_PATH, 10),
    verbose = 2,
    steps = 1
)

1/1 - 0s - 131ms/epoch - 131ms/step


array([[ 0.92518437,  0.02688943,  1.3779808 , -0.18135521,  0.28460196,
         0.09975183,  0.39145467,  0.10467056,  0.66134804,  1.0298098 ,
         1.0051594 ,  1.0250075 ,  0.33094838,  1.079399  ,  0.31543842,
         0.24437468],
       [ 0.9999299 ,  0.04703124,  1.0357064 ,  0.02062573,  0.13585581,
        -0.03668192,  0.24287906,  0.31435516,  0.66201687,  0.68312633,
         0.89222693,  1.0456262 ,  0.1873261 ,  0.8284791 ,  0.5529063 ,
         0.43569344],
       [ 0.7956152 ,  0.10995027,  0.891274  ,  0.17403986,  0.17465863,
        -0.18736658,  0.23635718,  0.24365589,  0.28401238,  0.6431897 ,
         0.99683315,  1.0461636 ,  0.15535057,  0.6205291 ,  0.3994919 ,
         0.43598735],
       [ 1.2121443 ,  0.18239702,  1.0053804 ,  0.15091293,  0.23851977,
        -0.31373733,  0.56532145, -0.1039922 ,  0.5700659 ,  0.54629576,
         0.9161004 ,  0.7502507 ,  0.4751567 ,  0.6888069 ,  0.4516763 ,
         0.36505973],
       [ 0.99018836,  0.18478835,  0

In [18]:
model.save('/content/drive/MyDrive/Colab Datasets/ResNet50-3DBB')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Datasets/ResNet50-3DBB/assets
